## Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import seaborn as sb
import pandas as pd

This specific notebook focuses on using MCMC methods to determine the coefficients of a polynomial.  In other words, today we seek to perform [Bayesian linear regression](https://en.wikipedia.org/wiki/Bayesian_linear_regression) using two different MCMC algorithms discussed in class, Metropolis-Hastings (MH), and Hybrid Monte-Carlo (HMC).  The former is considered the standard for MCMC algorithms and is often the basis upon which other MCMC algorithms have been developed.  Specifically here, HMC is relevant due to its utilization of gradients, which feature prominently in Neural Networks.  As such, HMC is particuluarly suited to adoption in machine learning problems.

## Data Generation

The function below generates points from a polynomial of a specified order.  The coefficients of this polynomial are chosen randomly between -10 and 10, and will be the features in the MCMC models experimented with.  The function returns both the point values, for plotting and training within the later MCMC models, and the generated coefficients for comparison to the results of the MCMC models.

In [ ]:
#generate data from a polynomial with random gaussian noise
def curve_gen(x,order,sigma_obs):
  coeffs = []
  #generate curve
  return_val = 0
  for exp in range(order+1):
    coeff = np.random.rand()*2 - 1
    coeffs.append(coeff) #store for checking results against later
    return_val += (coeff*(x**exp))

  #add randomness
  for i in range(len(return_val)):
    return_val[i] += np.random.normal(scale=sigma_obs)
  return return_val,coeffs

Below the order of the polynomial and number of points plotted can be chosen.  After this the function $\tt{curve\_gen}$ defined above is called and the data for later experimentation generated.  The coefficients are saved in the variable $\tt{act\_coeffs}$ for later comparison.  Note too that a seed is given to numpy to enable reproducability.  This line may be removed if a different result is desired.

In [ ]:
#for reproducability
np.random.seed(42)


# The order of the polynomial
order = 1

# The number of data points to simulate
num_pts = 10

# The x locations
x = np.linspace(-1,1,num_pts)

# The observation noise
sigma_obs = 0.1

y,act_coeffs = curve_gen(x,order,sigma_obs)

print(act_coeffs)

plt.plot(x,y,'.')
plt.show()

## Bayesian Functions
Here functions are defined for the calculation of the posterior distribution.  In particular we need both the prior and the likelihood for the calculation of the posterior.  According to Bayes' theorem, knowing these two terms along with the evidence (i.e. probability of the specific data being chosen) is all that is needed to calculate the posterior.  Bayes' theorem according to the variables defined in this document is presented below.

$$p(\beta|X,Y) = \cfrac{p(Y|X,𝜷)p(𝜷)}{p(Y|X)}$$

**Question: Why is there no equation for the evidence below?  Why is it not needed with MCMC methods?**

With linear regression (not only Bayesian linear regression), it's often useful to calculate a mathematical object known as the design matrix.

A general polynomial curve of order $i$ can be constructed as below, where $\beta_i$ is the coefficient on $x^i$.  Note that $\beta_0$ is the y-intercept.

$$y = \sum_{i=0}^i\beta_ix^i$$

Rather than using a sum, this equality can be equivalently written using vector multiplication.

$$y = 𝝓^T𝜷$$

$$𝝓 = \begin{pmatrix}
x^0 \\
x^1 \\
\vdots\\
x^i
\end{pmatrix} = 
\begin{pmatrix}
1 \\
x^1 \\
\vdots\\
x^i
\end{pmatrix}, 
𝜷 = \begin{pmatrix}
\beta_0 \\
\beta_1 \\
\vdots\\
\beta_i
\end{pmatrix}$$

In the case of multiple datapoints the vector $𝝓$ becomes the design matrix $𝜱$.

$$
𝜱 = 
\begin{pmatrix}
1 & x_0^1 & . . . & x_0^i\\
1 & x_1^1 & . . . & x_1^i\\
\vdots & \vdots & \ddots & \vdots\\
1 & x_i^1 & . . . & x_i^i
\end{pmatrix}
$$

Therefore, $Y = 𝜱𝜷$ in the case of multiple data points.  It is left to the student to write the function $\tt{design\_mat}$, which is intended to return a design matrix.  As well, the return function for the likelihood is left empty.  This needs to return the likelihood of $Y$ given $𝜷$ and $X$.  As a hint, it's useful to consult both the other functions compelted below as well as other online sources.  [This](https://alpopkes.com/posts/machine_learning/bayesian_linear_regression/) page, [this](https://letianzj.github.io/mcmc-linear-regression.html) page, and [this](https://letianzj.github.io/bayesian-linear-regression.html) page are recommended.  (Note: the design matrix is sometimes referred to as a Vandermond matrix; check out the numpy function vander).  

In [ ]:
def design_mat(x, order):
  #TODO: Write the function which will return the design matrix.
  return 

Now, we need to specify our models of the likelihood and prior distribution.  For the likelihood, you should use 
$$
P(Y|X,\beta) = \prod_{i=1}^n \frac{1}{\sigma \sqrt{2\pi}} \exp{-\frac{1}{2} \left( \frac{Y_i - X_i \beta}{\sigma} \right)^2},
$$
which has the logarithm
$$
\log P(Y|X,\beta) = -\frac{n}{2}(\ln(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^n (Y_i - X_i \beta)^2
$$
Note that this is just a simple sum of squares (with some extra constants).

For the prior, you should also assume that the parameters $\beta$ are, a priori, normally distributed with zero mean and given prior standard deviation $\sigma^2_{prior}$ 
$$
\log P(\beta_i) = -\frac{1}{2}(\ln(2\pi\sigma^2_{prior,i}) - \frac{1}{2\sigma^2_{prior,i}} \beta_i^2 .
$$
You might be able to convince yourself that this looks rather like L2 regularization.

Next, implement functions that accept $X$, $Y$, and $\beta$ and evaluate the log-likelihood, log-prior, and log-posterior (up to a constant of proportionality).  

In [ ]:

def log_likelihood(beta,x,y,sigma_obs):
  return

def log_prior(beta,sigma_prior):
  return 

def log_posterior(beta,x,y,sigma_obs,sigma_prior):
  log_like = log_likelihood(beta,x,y,sigma_obs)
  log_pr = log_prior(beta,sigma_prior)
  return log_like + log_pr


## Strongly suggest doing some unit testing here!

## The Algorithms!

With the Bayesian definitions above it's now time to define our algorithms.  The task today is to implement the Metropolis-Hastings algoirthm.  As a bonus you can try to implement Hybrid Monte-Carlo.  Very little code is provided for you.  While I was implementing these, the best source was the pseudo-code from the paper, followed closely by the links provided above.  However, if you get stuck feel free to ask one of us in class or take a look at the key provided!

**Question: Refer to Figure 5 in the paper, specifically the 5th line with the minimum function.  Is it necessary to have the proposal distribution q when calculating the acceptance rate?  If so why, if not why not?**

In [ ]:

#TODO: Implement Metropolis-Hastings algorithm
def mh(order,steps=10000,stepsize=0.1):
  betas = [np.zeros(order+1)] #initial guess
  for i in range(steps):
    if (i+1)%(steps/100) == 0:print("Step ",i+1,"/",steps,sep='') #print steps to check progress

    #get a random uniform sample, what should the bounds be?
    
    #there should be a new beta proposed in here
    
    #what should the acceptance look like?
    
    #do we need to look at the acceptance of all features individually or together?
  return betas

Below run your implemented Metropolis-Hastings.

In [ ]:
betas = mh(order)

Plot the results of your Metropolis-Hastings implementation.

Below is a histogram of your MCMC results along with a red line indicating the true coefficient value.

In [ ]:
fig, axes = plt.subplots(1, order+1)
fig.suptitle('Learned Distributions')
for i in range(len(axes)):
  axes[i].hist(np.array(betas)[:,i],density=True,range=(-1,1),bins=20)
  axes[i].axvline(x=act_coeffs[i],color='r')

Below a function $\tt{coeff\_plot}$ is provided which takes in a series of coefficients and returns y-values for plotting.  

In [ ]:
def coeff_plot(coeffs,x):
  #generate curve
  return_val = 0
  for exp in range(len(coeffs)):
    return_val += (coeffs[exp]*(x**exp))
  return return_val

First, **plot the random walk of MH algorithm throughout the feature space.** (i.e. plot the entire sequence of $\beta$ values as a scatter plot).

Next, **plot the posterior predictive distribution, i.e. make a plot of x versus y, overlaid with the predicted y for each sample of $\beta$ drawn via MCMC.

In [ ]:
plt.plot(x,y,'.')

plt.plot(x,coeff_plot(act_coeffs,x),label='Actual')

# for beta in samples, plot the regression line

plt.legend()

# Bonus: Hybrid MC

Before implementing HMC you will need to define the gradient of the posterior.  This is done for you.  However, the below function relies heavily on the gradient of the likelihood which is left mostly undefined.

Hint: What is the derivative of a normal distribution?  How do you account for the multiple features?  Do we already have easy access to such an object?

In [ ]:
def likelihood_grad(beta,x,y):
  #order is one less than the length of beta

  #what is the derivative of a normal distribution?
    
  #how are we going to account for gradient with respect to multiple features?

  return None

def posterior_grad(beta,x,y):
  like = likelihood_grad(beta,x,y)
  pr = prior(beta)
  return np.sum(like,axis=1)+pr

In [ ]:
#TODO: Implement Hybrid Monte-Carlo algorithm
def hmc(order, steps=10000,L=10,rho=0.1):
  betas = [np.zeros(order+1)]#initial guess
  for i in range(steps):
    if (i+1)%(steps/100) == 0:print("Step ",i+1,"/",steps,sep='')#print steps to check progress
    #now you need to sample from both a uniform and normal distribution
    
    #do you need a proposal distribution?  how are we going to get a proposed beta?
    
    #does anything special need to be done with rho?
    
    #how has the acceptance changed?
  return None

Below run your implemented Hybrid Monte-Carlo.

In [ ]:
betas = hmc(order)

Plot the random walk of HMC throughout the feature space.

In [ ]:
df = pd.DataFrame(betas)
g = sb.PairGrid(df)
g.map_offdiag(plt.scatter,s=10)
plt.show()

Compare the results of HMC to the actual coefficients used.

In [ ]:
plt.plot(x,y,'.')
plt.plot(x,coeff_plot(gen_coeffs(betas),x),label='MCMC')
plt.plot(x,coeff_plot(act_coeffs,x),label='Actual')
plt.legend()